In [1]:
import os
import gc
import sys
import json
import sqlite3
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from google.cloud import storage
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline

from geopy.geocoders import Nominatim
from pycep_correios import get_address_from_cep, WebService

sys.path.append(os.path.dirname(os.getcwd())+'/src')
import pipeline_modules

### Processing big data in chunks

In [20]:
# creating client list from items in storage
# creates json file in LOCAL - HAS TO BE UPLOADED TO STORAGE MANUALLY
#-----------------------------------------------------
def create_client_list(
    chunksize:int=2*10**7,
    initial_date:str='2021-10-01',
    prefix:str='gs://gpa-churn/',
    file_name:str='STAGING/relevanc_item_20220425.zip'
    ):
    
    ref_ids = []

    for chunk in pd.read_csv(
        prefix + file_name, 
        chunksize=chunksize, 
        usecols=['cod_cliente','dat_venda'],
        sep=';', 
        iterator=True
        ):

        chunk = chunk[chunk['dat_venda']>=initial_date]
        chunk.dropna(subset=['cod_cliente', 'dat_venda'], inplace=True)
        chunk['cod_cliente'] = chunk['cod_cliente'].astype(int)

        local_ids = list(chunk['cod_cliente'])
        ref_ids = ref_ids + local_ids
        ref_ids= list(set(ref_ids))

        print(f'len of ref_ids: {len(ref_ids)}')
        
    client_list = {
        'cod_cliente':ref_ids
    }
    
    with open('full_cod_cliente.json', 'w') as file:
        json.dump(client_list, file)
        
    return


# reading client list from storage
#-----------------------------------------------------
def read_client_list():
    
    with open('full_cod_cliente.json', 'r') as file:
        client_list = json.load(file)
    
    return client_list


# reading 'chunk' million lines as reference and searching for it's ids in the rest of the data
#-----------------------------------------------------
def get_items_from_codcliente(
    ref_ids:list,
    step:int=int(1*10**5),
    initial_date:str='2021-10-01',
    chunksize:int=50*10**6,
    prefix:str='gs://gpa-churn/',
    file_name:str='STAGING/relevanc_item_20220425.zip'
    ):
    
    columns = [
        'cod_cliente',
        'cod_loja',
        'dat_venda',
        'cod_interno_prod',
        'num_cupom',
        'val_venda_bruta_cupom',
        'qtd_item_venda'
        ]
    
    initial_dtypes = {
        'cod_cliente':'float32',
        'cod_loja':'float32',
        'data_venda':'str',
        'cupom':'float32',
        'cod_produto':'float32',
        'val_venda_bruta_cupom':'float32',
        'qtd_item_venda':'float32'
        }
    
    n_steps = np.ceil(len(ref_ids)/step)
    step_count = 0
    while step_count < n_steps:
        
        print('-'*20)
        print('STEP: ', step_count)
        id_list = ref_ids[int(step_count*step):int((step_count+1)*step)]
    
        df_list = []
        n_rows = 0
        chunk_number = 0
        for chunk in pd.read_csv(
            prefix + file_name, 
            chunksize=chunksize, 
            usecols=columns,
            dtype=initial_dtypes,
            sep=';', 
            iterator=True
            ):

            chunk = chunk[chunk['dat_venda']>=initial_date]
            chunk.dropna(subset=['cod_cliente', 'dat_venda'], inplace=True)
            chunk['cod_cliente'] = chunk['cod_cliente'].astype(int)

            chunk = chunk[chunk['cod_cliente'].isin(id_list)]


            chunk_number = chunk_number + 1
            n_rows = n_rows + len(chunk)
            df_list.append(chunk)
            print(f'Processed {chunk_number} chunks, number of rows: {n_rows}')

        df = pd.concat(df_list, axis=0)
        del df_list, chunk

        items_cols_to_rename = {
            'cod_interno_prod':'cod_produto',
            'num_cupom':'cupom',
            'dat_venda':'data_venda'
        }
        df.rename(columns=items_cols_to_rename, inplace=True)
        df = df.drop_duplicates()
        df.reset_index(drop=True, inplace=True)

        dtypes = {
            'cod_cliente':'int32',
            'cod_loja':'int16',
            'data_venda':'str',
            'cupom':'int32',
            'cod_produto':'int32',
            'val_venda_bruta_cupom':'float32',
            'qtd_item_venda':'int8'
            }
        for var, tp in dtypes.items():
            df[var] = df[var].astype(tp)
            
        df.to_parquet(f'gs://gpa-churn/data/raw/items/items_{step_count}.parquet', compression='gzip')
        step_count = step_count+1
    
    return


# getting activations from ref_ids - obtained from items' df
#-----------------------------------------------------
def get_activations_from_codcliente(
    ref_ids:list,
    step:int=int(1*10**5),
    chunksize:int=int(20*10**6),
    prefix:str='gs://gpa-churn/',
    file_name:str='STAGING/V_DW_FM16_MOV_VEND_CRM.TXT.zip'
    ):
    
    names=[
        'unknown',
        'cupom',
        'cod_produto',
        'cod_loja',
        'cod_cliente',
        'val_gross_margin_cupom',
        'val_vend_bruta_mercad',
        'flg_vend_meu_desct',
        'valor_desconto',
        'tipo_promo',
        'flag_dev'
    ]
    
    columns = [
        'cupom',
        'cod_produto',
        'cod_loja',
        'cod_cliente',
        'val_gross_margin_cupom',
        'val_vend_bruta_mercad',
        'flg_vend_meu_desct',
        'valor_desconto',
        'tipo_promo',
        'flag_dev'
    ]
    
    dtypes = {
        'cupom':'int32',
        'cod_produto':'int32',
        'cod_loja':'int16',
        'cod_cliente':'int32',
        'val_gross_margin_cupom':'float32',
        'val_vend_bruta_mercad':'float32',
        'flg_vend_meu_desct':'int8',
        'valor_desconto':'float32',
        'tipo_promo':'str',
        'flag_dev':'int8'
    }
    
    n_steps = np.ceil(len(ref_ids)/step)
    step_count = 0
    while step_count < n_steps:
    
        print('-'*20)
        print('STEP: ', step_count)
        id_list = ref_ids[int(step_count*step):int((step_count+1)*step)]
    
        df_list = []
        n_rows = 0
        chunk_number = 0
        for chunk in pd.read_csv(
            prefix + file_name,
            chunksize=chunksize, 
            sep=';',
            names=names,
            usecols=columns,
            dtype=dtypes,
            iterator=True
            ):

            chunk = chunk[chunk['cod_cliente'].isin(id_list)]
            chunk = chunk.drop_duplicates()
            df_list.append(chunk)

            chunk_number = chunk_number + 1
            n_rows = n_rows + len(chunk)
            print(f'Processed {chunk_number} chunks, number of rows: {n_rows}')

        df = pd.concat(df_list, axis=0)
        del df_list, chunk
        df = df.drop_duplicates()
        
        df.to_parquet(f'gs://gpa-churn/data/raw/activations/activations_{step_count}.parquet', compression='gzip')
        step_count = step_count+1
    
    return

In [24]:
# get_items_from_codcliente(
#     ref_ids=ref_ids
#     )

In [25]:
# client_list = read_client_list()
# type(client_list)

In [26]:
# get_activations_from_codcliente(
#     ref_ids=client_list['cod_cliente']
#     )

### Basic functions

In [3]:
def read_files_from_staging(
    obj_list:list
    ):
    
    df_list = []
    for file in obj_list:
        try:
            local_df = pd.read_csv(prefix + file, sep=';')
        except:
            local_df = pd.read_csv(prefix + file, sep='\n')
            column_list = local_df.columns[0].split(';')
            local_df = local_df[local_df.columns[0]].str.split(';', expand=True)
            local_df.columns = column_list
        df_list.append(local_df)
        print('added file: ', file)
    print('-'*10)
    df = pd.concat(df_list, axis=0)
    df = df.drop_duplicates()
    df.reset_index(drop=True, inplace=True)
    
    return df


def get_products_data(
    tables_dict:dict
    ):
    
    products = read_files_from_staging(
        tables_dict['products']
        )

    products = products[products['ind_ativo']==True]
    products_cols_to_drop = [
        'cod_subcategoria',
        'cod_categoria',
        'desc_subcategoria',
        'desc_subcategoria',
        'cod_grupo',
        'desc_grupo',
        'cod_subgrupo',
        'desc_subgrupo',
        'cod_departamento',
        'ind_ativo',
        'desc_plu' # product names - comment to validate
    ]
    products.drop(columns=products_cols_to_drop, inplace=True)

    products_cols_to_rename = {
        'cod_plu':'cod_produto',
        #'desc_plu':'produto', # product names - uncomment to validate
        'desc_categoria':'categoria',
        'desc_departamento':'departamento'
    }
    products.rename(columns=products_cols_to_rename, inplace=True)

    products_cols_to_lower = ['categoria', 'departamento'] #,'produto'] # product names - uncomment to validate
    for var in products_cols_to_lower:
        products[var] = products[var].str.lower()
    
    return products


def get_register_data(
    tables_dict:dict
    ):
    
    register = read_files_from_staging(
        tables_dict['register']
        )

    register.sort_values(by=['idcliente','datultatual'], inplace=True, ascending=True)
    register.drop_duplicates(subset=['idcliente','datultatual'], keep='last')

    register_cols_to_drop = ['codorigcliente','codorigcliente']
    register = register.drop(columns=register_cols_to_drop)
    register = register.dropna(subset=['idcliente','cidadecli','ufcli'], how='any', axis=0)

    register_cols_to_rename = {
        'idcliente':'cod_cliente',
        'datcadorigem':'data_cadastro',
        'codsexo':'sexo',
        'cidadecli':'cidade',
        'ufcli':'uf',
        'datnasccli':'data_nascimento',
        'datultatual':'data_registro'
    }
    register.rename(columns=register_cols_to_rename, inplace=True)

    register_cols_to_lower = ['cidade', 'uf']
    for var in register_cols_to_lower:
        register[var] = register[var].str.lower()

    register = register.drop_duplicates(subset=['cod_cliente','sexo','cidade','uf','data_nascimento'])
    register.reset_index(drop=True, inplace=True)
    
    return register


def get_stix_data(
    tables_dict:dict
    ):
    
    stix = read_files_from_staging(
        tables_dict['stix']
        )
    
    stix.drop(columns=['dt_nascimento'], inplace=True)
    stix.sort_values(by=['id_cliente','dt_cadastro_origem'], ascending=True, inplace=True)
    stix = stix.drop_duplicates(subset=['id_cliente','ind_email'], keep='first')
    stix['ind_email'] = abs(stix['ind_email']-2)
    stix.reset_index(drop=True, inplace=True)
    
    stix_cols_to_rename = {
        'id_cliente':'cod_cliente',
        'dt_cadastro_origem':'data_registro_stix'
    }
    stix.rename(columns=stix_cols_to_rename, inplace=True)
    
    return stix

### Prepare samples from items and activations data

This method proved to be very misleading.

In [4]:
# # reading 'chunk' million lines as reference and searching for it's ids in the rest of the data
# #-----------------------------------------------------
# def get_items_dataframe(
#     initial_date:str='2021-10-01',
#     chunksize:int=0.5*10**6,
#     prefix:str='gs://gpa-churn/',
#     file_name:str='STAGING/relevanc_item_20220425.zip'
#     ):
    
#     columns = [
#         'cod_cliente',
#         'cod_loja',
#         'dat_venda',
#         'cod_interno_prod',
#         'num_cupom',
#         'val_venda_bruta_cupom',
#         'qtd_item_venda'
#         ]
    
#     dtypes = {
#         'cod_cliente':'float32',
#         'cod_loja':'float32',
#         'data_venda':'str',
#         'cupom':'float32',
#         'cod_produto':'float32',
#         'val_venda_bruta_cupom':'float32',
#         'qtd_item_venda':'float32'
#         }
    
#     df_list = []
#     n_rows = 0
#     chunk_number = 0
#     for chunk in pd.read_csv(
#         prefix + file_name, 
#         chunksize=chunksize, 
#         usecols=columns,
#         dtype=dtypes,
#         sep=';', 
#         iterator=True
#         ):
        
#         chunk = chunk[chunk['dat_venda']>=initial_date]
#         chunk.dropna(subset=['cod_cliente', 'dat_venda'], inplace=True)
#         chunk['cod_cliente'] = chunk['cod_cliente'].astype(int)
        
#         if chunk_number==0:
#             ref_ids = set(chunk['cod_cliente'])
#             print(f'len of ref_ids: {len(ref_ids)}')
#         else:
#             chunk = chunk[chunk['cod_cliente'].isin(ref_ids)]
        
        
#         chunk_number = chunk_number + 1
#         n_rows = n_rows + len(chunk)
#         df_list.append(chunk)
        
#         if (chunk_number % 20) == 0:
#             print(f'Processed {chunk_number} chunks, number of rows: {n_rows}')
        
#     df = pd.concat(df_list, axis=0)
#     del df_list, chunk
    
#     items_cols_to_rename = {
#         'cod_interno_prod':'cod_produto',
#         'num_cupom':'cupom',
#         'dat_venda':'data_venda'
#     }
#     df.rename(columns=items_cols_to_rename, inplace=True)
#     df = df.drop_duplicates()
#     df.reset_index(drop=True, inplace=True)
    
#     dtypes = {
#         'cod_cliente':'int32',
#         'cod_loja':'int16',
#         'data_venda':'str',
#         'cupom':'int32',
#         'cod_produto':'int32',
#         'val_venda_bruta_cupom':'float32',
#         'qtd_item_venda':'int8'
#         }
#     for var, tp in dtypes.items():
#         df[var] = df[var].astype(tp)
    
#     return df, ref_ids


# # Running functions
# #-----------------------------------------------------

# items, ref_ids = get_items_dataframe()
# items.to_parquet('gs://gpa-churn/data/raw/items/items.parquet', compression='gzip')
# print(items.head())

In [5]:
# # getting activations from ref_ids - obtained from items' df
# #-----------------------------------------------------
# def match_activations_dataframe(
#     ref_ids:list,
#     chunksize:int=15*10**6,
#     prefix:str='gs://gpa-churn/',
#     file_name:str='STAGING/V_DW_FM16_MOV_VEND_CRM.TXT.zip'
#     ):
    
#     names=[
#         'unknown',
#         'cupom',
#         'cod_produto',
#         'cod_loja',
#         'cod_cliente',
#         'val_gross_margin_cupom',
#         'val_vend_bruta_mercad',
#         'flg_vend_meu_desct',
#         'valor_desconto',
#         'tipo_promo',
#         'flag_dev'
#     ]
    
#     columns = [
#         'cupom',
#         'cod_produto',
#         'cod_loja',
#         'cod_cliente',
#         'val_gross_margin_cupom',
#         'val_vend_bruta_mercad',
#         'flg_vend_meu_desct',
#         'valor_desconto',
#         'tipo_promo',
#         'flag_dev'
#     ]
    
#     dtypes = {
#         'cupom':'int32',
#         'cod_produto':'int32',
#         'cod_loja':'int16',
#         'cod_cliente':'int32',
#         'val_gross_margin_cupom':'float32',
#         'val_vend_bruta_mercad':'float32',
#         'flg_vend_meu_desct':'int8',
#         'valor_desconto':'float32',
#         'tipo_promo':'str',
#         'flag_dev':'int8'
#     }
    
#     chunk_number = 0
#     df_list = []
#     for chunk in pd.read_csv(
#         prefix + file_name,
#         chunksize=chunksize, 
#         sep=';',
#         names=names,
#         usecols=columns,
#         dtype=dtypes,
#         iterator=True
#         ):
        
#         chunk = chunk[chunk['cod_cliente'].isin(ref_ids)]
#         chunk = chunk.drop_duplicates()
#         df_list.append(chunk)
        
#         chunk_number = chunk_number + 1
#         print(f'Processed {chunk_number} chunks')
        
#     df = pd.concat(df_list, axis=0)
#     del df_list, chunk
#     df = df.drop_duplicates()
    
#     return df


# # Running functions
# #-----------------------------------------------------

# activations = match_activations_dataframe(ref_ids=ref_ids)
# activations.to_parquet('gs://gpa-churn/data/raw/activations/activations.parquet', compression='gzip')
# print(activations.head())

## Reading preprocessed items and activations

In [6]:
n_clients = int(1*10**5)

In [7]:
items = pd.read_parquet('gs://gpa-churn/data/raw/items/items.parquet')
items = items[items['data_venda']>='2021-10-01']
client_list = list(set(items['cod_cliente']))
print(f'original number of clients: {len(client_list)}')
items = items[items['cod_cliente'].isin(client_list[0:n_clients])]
print(f'adjusted number of clients: {len(client_list[0:n_clients])}')
items.info()

original number of clients: 275473
adjusted number of clients: 100000
<class 'pandas.core.frame.DataFrame'>
Int64Index: 31780804 entries, 1 to 87584398
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   cod_cliente            int32  
 1   cod_loja               int16  
 2   data_venda             object 
 3   cupom                  int32  
 4   cod_produto            int32  
 5   val_venda_bruta_cupom  float32
 6   qtd_item_venda         int8   
dtypes: float32(1), int16(1), int32(3), int8(1), object(1)
memory usage: 1.0+ GB


In [9]:
df = pd.merge(items, activations, on=['cod_cliente','cupom','cod_loja','cod_produto'], how='outer')
del items, activations, client_list
gc.collect()

dropna_list = ['data_venda']
df = df.dropna(subset=dropna_list)

fillna_numerical = ['val_venda_bruta_cupom','qtd_item_venda','flg_vend_meu_desct', 'valor_desconto', 'flag_dev']
fillna_categorical = ['tipo_promo']

df[fillna_numerical] = df[fillna_numerical].fillna(0)
df[fillna_categorical] = df[fillna_categorical].fillna('missing')

convert_dtypes = {
    'val_venda_bruta_cupom':'float32',
    'qtd_item_venda':'float32',
    'flg_vend_meu_desct':'int8',
    'valor_desconto':'float32',
    'flag_dev':'int8'
}

for var, tp in convert_dtypes.items():
    df[var] = df[var].astype(tp)

df

,cod_cliente,cod_loja,data_venda,cupom,cod_produto,val_venda_bruta_cupom,qtd_item_venda,flg_vend_meu_desct,valor_desconto,tipo_promo,flag_dev
0,13218657,5470,2022-02-16,12344614,1282979,6.59,1.0,0,0.00,missing,0
1,21599310,2438,2022-02-17,12344250,249294,5.59,4.0,0,0.00,missing,0
2,16967032,5470,2022-02-16,12345756,1148447,4.37,2.0,0,0.00,missing,0
3,9546587,5470,2022-02-16,12345610,1222353,6.49,2.0,0,0.00,missing,0
4,46269972,5470,2022-02-16,12344989,350464,19.90,3.0,0,0.00,missing,0
...,...,...,...,...,...,...,...,...,...,...,...
31784258,12645288,2457,2022-02-15,323655,4625971,11.79,1.0,0,0.00,X,0
31784259,54172272,205,2022-02-15,350288,250474,9.99,0.0,0,0.00,missing,0
31784260,25242848,1223,2022-02-15,455771,248334,4.19,1.0,0,0.00,missing,0
31784261,23757426,1516,2022-02-15,148999,5029,17.99,0.0,1,1.81,G,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31784263 entries, 0 to 31784262
Data columns (total 11 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   cod_cliente            int32  
 1   cod_loja               int16  
 2   data_venda             object 
 3   cupom                  int32  
 4   cod_produto            int32  
 5   val_venda_bruta_cupom  float32
 6   qtd_item_venda         float32
 7   flg_vend_meu_desct     int8   
 8   valor_desconto         float32
 9   tipo_promo             object 
 10  flag_dev               int8   
dtypes: float32(3), int16(1), int32(3), int8(2), object(2)
memory usage: 1.5+ GB


In [11]:
pipe_encode_promo = Pipeline([
    ('group_rare_promo', pipeline_modules.group_rare_categorical(columns=['tipo_promo'], threshold=0.002)),
    ('encode_promo', pipeline_modules.encode_categorical(columns=['tipo_promo']))
])

df = pipe_encode_promo.fit_transform(df)

columns_to_convert = [i for i in list(df.columns) if 'tipo_promo' in i]
convert_dtypes = {key:'int8' for key in columns_to_convert}
for var, tp in convert_dtypes.items():
    df[var] = df[var].astype(tp)
    
df

Some variables were not available.
If haven't removed features in other process, might have entered a bad string.
Some variables were not available.
If haven't removed features in other process, might have entered a bad string.


,cod_cliente,cod_loja,data_venda,cupom,cod_produto,val_venda_bruta_cupom,qtd_item_venda,flg_vend_meu_desct,valor_desconto,flag_dev,tipo_promo_0,tipo_promo_1,tipo_promo_2,tipo_promo_3,tipo_promo_4,tipo_promo_5
0,13218657,5470,2022-02-16,12344614,1282979,6.59,1.0,0,0.00,0,0,0,0,0,0,1
1,21599310,2438,2022-02-17,12344250,249294,5.59,4.0,0,0.00,0,0,0,0,0,0,1
2,16967032,5470,2022-02-16,12345756,1148447,4.37,2.0,0,0.00,0,0,0,0,0,0,1
3,9546587,5470,2022-02-16,12345610,1222353,6.49,2.0,0,0.00,0,0,0,0,0,0,1
4,46269972,5470,2022-02-16,12344989,350464,19.90,3.0,0,0.00,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31784258,12645288,2457,2022-02-15,323655,4625971,11.79,1.0,0,0.00,0,0,0,0,1,0,0
31784259,54172272,205,2022-02-15,350288,250474,9.99,0.0,0,0.00,0,0,0,0,0,0,1
31784260,25242848,1223,2022-02-15,455771,248334,4.19,1.0,0,0.00,0,0,0,0,0,0,1
31784261,23757426,1516,2022-02-15,148999,5029,17.99,0.0,1,1.81,0,1,0,0,0,0,0


## Listing input files

In [12]:
prefix = 'gs://gpa-churn/'

In [13]:
# listing objects in blob storage bucket
storage_client = storage.Client()
obj_list = storage_client.list_blobs('gpa-churn')
obj_list = [i.name for i in obj_list if 'STAGING/' in i.name]

tables_dict = {
    'register':[i for i in obj_list if 'cadastro' in i],
    'stix':[i for i in obj_list if 'stix_optin' in i],
    'email':[i for i in obj_list if 'optin_email' in i],
    'items':[i for i in obj_list if 'relevanc_item' in i],
    'stores':[i for i in obj_list if 'relevanc_store' in i],
    'products':[i for i in obj_list if 'relevanc_product' in i],
    'activations':[i for i in obj_list if 'mov_vend_crm' in i]
}

## Merging sales (items and activations) and product information

In [14]:
products = get_products_data(tables_dict)

pipe_encode_products = Pipeline([
    ('group_rare_products', pipeline_modules.group_rare_categorical(columns=['categoria','departamento'])),
    ('encode_products', pipeline_modules.encode_categorical(columns=['categoria','departamento']))
])

products = pipe_encode_products.fit_transform(products)

columns_to_convert_1 = [i for i in list(products.columns) if 'categoria' in i]
columns_to_convert_2 = [i for i in list(products.columns) if 'departamento' in i]
columns_to_convert = columns_to_convert_1 + columns_to_convert_2
del columns_to_convert_1, columns_to_convert_2
gc.collect()

convert_dtypes = {key:'int8' for key in columns_to_convert}
for var, tp in convert_dtypes.items():
    products[var] = products[var].astype(tp)

products.head()

added file:  STAGING/relevanc_product_20220425.zip
----------
Some variables were not available.
If haven't removed features in other process, might have entered a bad string.
Some variables were not available.
If haven't removed features in other process, might have entered a bad string.


,cod_produto,categoria_0,categoria_1,categoria_2,categoria_3,categoria_4,categoria_5,categoria_6,categoria_7,departamento_0
0,833,1,0,0,0,0,0,0,0,1
1,2233,0,0,0,0,0,1,0,0,1
2,6033,0,0,0,0,0,1,0,0,1
3,8433,0,0,0,0,0,0,0,0,1
4,11433,0,0,0,0,0,0,1,0,1


In [15]:
df = pd.merge(df, products, on=['cod_produto'])
del products
gc.collect()
df.drop(columns=['cod_produto'], inplace=True)
df.head()

,cod_cliente,cod_loja,data_venda,cupom,val_venda_bruta_cupom,qtd_item_venda,flg_vend_meu_desct,valor_desconto,flag_dev,tipo_promo_0,...,tipo_promo_5,categoria_0,categoria_1,categoria_2,categoria_3,categoria_4,categoria_5,categoria_6,categoria_7,departamento_0
0,13218657,5470,2022-02-16,12344614,6.59,1.0,0,0.0,0,0,...,1,0,0,0,0,1,0,0,0,0
1,51384928,5470,2021-12-13,11955754,8.49,1.0,0,0.0,0,0,...,1,0,0,0,0,1,0,0,0,0
2,11097762,9,2022-02-18,12346370,6.59,2.0,0,0.0,0,0,...,1,0,0,0,0,1,0,0,0,0
3,3801731,1221,2022-03-15,466464,7.99,2.0,0,0.0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,10050749,2380,2022-02-18,12353754,5.93,2.0,0,0.0,0,0,...,1,0,0,0,0,1,0,0,0,0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29755561 entries, 0 to 29755560
Data columns (total 24 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   cod_cliente            int32  
 1   cod_loja               int16  
 2   data_venda             object 
 3   cupom                  int32  
 4   val_venda_bruta_cupom  float32
 5   qtd_item_venda         float32
 6   flg_vend_meu_desct     int8   
 7   valor_desconto         float32
 8   flag_dev               int8   
 9   tipo_promo_0           int8   
 10  tipo_promo_1           int8   
 11  tipo_promo_2           int8   
 12  tipo_promo_3           int8   
 13  tipo_promo_4           int8   
 14  tipo_promo_5           int8   
 15  categoria_0            int8   
 16  categoria_1            int8   
 17  categoria_2            int8   
 18  categoria_3            int8   
 19  categoria_4            int8   
 20  categoria_5            int8   
 21  categoria_6            int8   
 22  categoria_7     

It's convenient to create aggregation features at this point. We can create features of the last 3 months, (sum or individual values) for each combination of [cod_cliente,data_venda].

In [17]:
df[['ano','mes','dia']] = df['data_venda'].str.split('-', expand=True)
df.drop(columns=['dia'], inplace=True)
df

,cod_cliente,cod_loja,data_venda,cupom,val_venda_bruta_cupom,qtd_item_venda,flg_vend_meu_desct,valor_desconto,flag_dev,tipo_promo_0,...,categoria_1,categoria_2,categoria_3,categoria_4,categoria_5,categoria_6,categoria_7,departamento_0,ano,mes
0,13218657,5470,2022-02-16,12344614,6.590000,1.0,0,0.0,0,0,...,0,0,0,1,0,0,0,0,2022,02
1,51384928,5470,2021-12-13,11955754,8.490000,1.0,0,0.0,0,0,...,0,0,0,1,0,0,0,0,2021,12
2,11097762,9,2022-02-18,12346370,6.590000,2.0,0,0.0,0,0,...,0,0,0,1,0,0,0,0,2022,02
3,3801731,1221,2022-03-15,466464,7.990000,2.0,0,0.0,0,0,...,0,0,0,1,0,0,0,0,2022,03
4,10050749,2380,2022-02-18,12353754,5.930000,2.0,0,0.0,0,0,...,0,0,0,1,0,0,0,0,2022,02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29755556,15877355,11,2022-02-23,12396115,150.399994,1.0,0,0.0,0,0,...,0,1,0,0,0,0,0,0,2022,02
29755557,3776043,2435,2022-03-03,637584,5.790000,2.0,0,0.0,0,0,...,0,0,0,1,0,0,0,0,2022,03
29755558,10653187,2836,2021-12-30,145846,4.990000,1.0,0,0.0,0,0,...,0,0,0,0,0,0,1,0,2021,12
29755559,45231200,2369,2021-12-04,329504,8.690000,1.0,0,0.0,0,0,...,0,0,0,0,1,0,0,1,2021,12


In [18]:
df.to_parquet('gs://gpa-churn/data/raw/reshape/reshape_110k.parquet', compression='gzip')

### Creating aggregations

Aggregations must comprehend window of '2021-10-01' - '2022-04-2X'

In [19]:
# df = pd.read_parquet('gs://gpa-churn/data/raw/reshape/reshape_100k.parquet')
# df.info()

In [20]:
# df[(df['ano']=='2022') & (df['mes']=='05')]['flg_vend_meu_desct'].value_counts()

In [21]:
# df[(df['cod_cliente']==18) & (df['mes']=='12')].head(60)

In [22]:
sum_m0 = df.drop(columns=['cod_loja','data_venda','cupom']).copy()
sum_m0 = sum_m0.groupby(['cod_cliente','ano','mes']).sum()

rename_dict = {
    'val_venda_bruta_cupom': 'total_val_venda_bruta_cupom', #valor total gasto no mês
    'qtd_item_venda': 'total_qtd_item_venda', #total de itens comprados no mês
    'valor_desconto': 'total_valor_desconto', #valor total de desconto no mês
    'flg_vend_meu_desct' : 'total_flg_vend_meu_desct',
    'flag_dev' : 'total_flag_dev',
    'tipo_promo_0' : 'total_tipo_promo_0',
    'tipo_promo_1' : 'total_tipo_promo_1',
    'tipo_promo_2' : 'total_tipo_promo_2',
    'tipo_promo_3' : 'total_tipo_promo_3',
    'tipo_promo_4' : 'total_tipo_promo_4',
    'tipo_promo_5' : 'total_tipo_promo_5',
    'categoria_0': 'total_categoria_0',
    'categoria_1': 'total_categoria_1',
    'categoria_2': 'total_categoria_2',
    'categoria_3': 'total_categoria_3',
    'categoria_4': 'total_categoria_4',
    'categoria_5': 'total_categoria_5',
    'categoria_6': 'total_categoria_6',
    'categoria_7': 'total_categoria_7',
    'departamento_0': 'total_departamento_0'
}
sum_m0 = sum_m0.rename(columns=rename_dict)
sum_m0.reset_index(inplace=True)
sum_m0

,cod_cliente,ano,mes,total_val_venda_bruta_cupom,total_qtd_item_venda,total_flg_vend_meu_desct,total_valor_desconto,total_flag_dev,total_tipo_promo_0,total_tipo_promo_1,...,total_tipo_promo_5,total_categoria_0,total_categoria_1,total_categoria_2,total_categoria_3,total_categoria_4,total_categoria_5,total_categoria_6,total_categoria_7,total_departamento_0
0,18,2021,12,597.210022,30.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,1.0,0.0,0.0,1.0,9.0,4.0,2.0,27.0
1,18,2022,01,11.590000,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,18,2022,03,327.040009,22.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,8.0,1.0,8.0,3.0,0.0,15.0
3,18,2022,04,475.600006,18.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,1.0,3.0,0.0,7.0,2.0,0.0,16.0
4,65,2021,10,197.490005,22.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,3.0,3.0,6.0,4.0,1.0,0.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575272,54306812,2022,02,313.169983,16.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7.0,0.0,1.0,0.0,4.0,2.0,0.0,9.0
575273,54306812,2022,03,184.279999,20.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,3.0,2.0,11.0,1.0,0.0,12.0
575274,54319272,2022,02,2058.119873,168.0,0.0,0.0,0.0,0.0,0.0,...,138.0,5.0,3.0,40.0,17.0,4.0,7.0,30.0,2.0,72.0
575275,54319272,2022,03,931.830017,111.0,0.0,0.0,0.0,0.0,0.0,...,78.0,0.0,3.0,28.0,16.0,8.0,1.0,9.0,0.0,23.0


In [23]:
count_aux_df = df[['cod_cliente','cupom','ano','mes']]

count_m0 = count_aux_df.groupby(['cod_cliente','ano','mes']).count()
rename_dict = {
    'cupom': 'total_compras_mes'
}
count_m0 = count_m0.rename(columns=rename_dict)
count_m0.reset_index(inplace=True)
count_m0

,cod_cliente,ano,mes,total_compras_mes
0,18,2021,12,31
1,18,2022,01,2
2,18,2022,03,25
3,18,2022,04,20
4,65,2021,10,21
...,...,...,...,...
575272,54306812,2022,02,17
575273,54306812,2022,03,21
575274,54319272,2022,02,138
575275,54319272,2022,03,78


In [24]:
segmented_df = pd.merge(sum_m0, count_m0, on=['cod_cliente','ano','mes'])
del sum_m0, count_m0
gc.collect()
segmented_df.head()

,cod_cliente,ano,mes,total_val_venda_bruta_cupom,total_qtd_item_venda,total_flg_vend_meu_desct,total_valor_desconto,total_flag_dev,total_tipo_promo_0,total_tipo_promo_1,...,total_categoria_0,total_categoria_1,total_categoria_2,total_categoria_3,total_categoria_4,total_categoria_5,total_categoria_6,total_categoria_7,total_departamento_0,total_compras_mes
0,18,2021,12,597.210022,30.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,0.0,0.0,1.0,9.0,4.0,2.0,27.0,31
1,18,2022,01,11.590000,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2
2,18,2022,03,327.040009,22.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,8.0,1.0,8.0,3.0,0.0,15.0,25
3,18,2022,04,475.600006,18.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,1.0,3.0,0.0,7.0,2.0,0.0,16.0,20
4,65,2021,10,197.490005,22.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,3.0,3.0,6.0,4.0,1.0,0.0,8.0,21


### Create date columns

We must form chunks of dates - three months past and foward. The foward is used to create the target, the past is used as independent features.

In [25]:
segmented_df['date'] = segmented_df[['ano','mes']].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
segmented_df['date'] = pd.to_datetime(segmented_df['date'], format='%Y-%m')
segmented_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 575277 entries, 0 to 575276
Data columns (total 25 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   cod_cliente                  575277 non-null  int64         
 1   ano                          575277 non-null  object        
 2   mes                          575277 non-null  object        
 3   total_val_venda_bruta_cupom  575277 non-null  float32       
 4   total_qtd_item_venda         575277 non-null  float32       
 5   total_flg_vend_meu_desct     575277 non-null  float64       
 6   total_valor_desconto         575277 non-null  float32       
 7   total_flag_dev               575277 non-null  float64       
 8   total_tipo_promo_0           575277 non-null  float64       
 9   total_tipo_promo_1           575277 non-null  float64       
 10  total_tipo_promo_2           575277 non-null  float64       
 11  total_tipo_promo_3        

In [26]:
# segmented_df.to_parquet('gs://gpa-churn/data/raw/segmented/segmented_100k.parquet', compression='gzip')

### Creating samples

Samples are created using time filters.

In [27]:
segmented_df = pd.read_parquet('gs://gpa-churn/data/raw/segmented/segmented_100k.parquet')

In [28]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

initial_date = '2021-10-01'
initial_date = datetime.strptime('2021-10-01', '%Y-%m-%d')
final_date = segmented_df['date'].max()
print(f'initial date: {initial_date}')
print(f'final date: {final_date}')

total_time_delta = final_date - initial_date
n_months = np.ceil(total_time_delta/np.timedelta64(1, 'M')) + 1
n_samples = (n_months-6)+1
print(f'total span in months: {n_months}')
print(f'number of samples generated: {n_samples}')

unitary_increment = relativedelta(months=+1)
print(f'unitary_increment: {unitary_increment}')

sample_range = relativedelta(months=+2)
print(f'sample_range: {sample_range}')

initial date: 2021-10-01 00:00:00
final date: 2022-04-01 00:00:00
total span in months: 7.0
number of samples generated: 2.0
unitary_increment: relativedelta(months=+1)
sample_range: relativedelta(months=+2)


In [29]:
print('Generating samples from segmented dataframe')
print('#'*10)
current_date = initial_date+sample_range
samples_list = []
step = 0
while step < n_samples:
    
    print('REFERENCE DATE: ', current_date)
    
    lower_mask = segmented_df['date']>=current_date-sample_range
    upper_mask = segmented_df['date']<=current_date
    features_df = segmented_df[lower_mask & upper_mask]
    features_df = features_df.sort_values(by=['cod_cliente','date'])
    print('features:')
    print(features_df['date'].value_counts())
    
    lower_mask = segmented_df['date']>=current_date+unitary_increment
    upper_mask = segmented_df['date']<=current_date+unitary_increment+sample_range
    target_df = segmented_df[lower_mask & upper_mask]
    print('target:')
    print(target_df['date'].value_counts())
    target_df = target_df.sort_values(by=['cod_cliente','date'])
    target_df.rename(columns={'total_qtd_item_venda':'target'}, inplace=True)
    target_df = target_df[['cod_cliente','target']].groupby(['cod_cliente']).sum()
    
    current_month_mask = segmented_df['date']==current_date
    features_m0 = features_df[current_month_mask].drop(columns=['date','ano','mes'])
    
    features_m3 = features_df.drop(columns=['date','ano','mes']).groupby(['cod_cliente']).sum()
    features_m3.columns = ['avg_l3m_' + i for i in list(features_m3.columns)]
    features_m3.reset_index(inplace=True)
    
    sample = pd.merge(features_m0, features_m3, on='cod_cliente')
    sample = pd.merge(sample, target_df, on='cod_cliente', how='left')
    sample['date'] = current_date
    sample['target'] = [0 if i>0 else 1 for i in list(sample['target'])]
    samples_list.append(sample)
    
    current_date+=unitary_increment
    step+=1
    
samples_df = pd.concat(samples_list)
samples_df.drop_duplicates(inplace=True)
print('#'*10)
print('Successfully generated samples')
samples_df.head()

Generating samples from segmented dataframe
##########
REFERENCE DATE:  2021-12-01 00:00:00
features:
2021-12-01    86225
2021-11-01    84799
2021-10-01    66743
Name: date, dtype: int64
target:
2022-03-01    85753
2022-01-01    85712
2022-02-01    84729
Name: date, dtype: int64


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


REFERENCE DATE:  2022-01-01 00:00:00
features:
2021-12-01    86225
2022-01-01    85712
2021-11-01    84799
Name: date, dtype: int64
target:
2022-03-01    85753
2022-02-01    84729
2022-04-01    81316
Name: date, dtype: int64
##########
Successfully generated samples


,cod_cliente,total_val_venda_bruta_cupom,total_qtd_item_venda,total_flg_vend_meu_desct,total_valor_desconto,total_flag_dev,total_tipo_promo_0,total_tipo_promo_1,total_tipo_promo_2,total_tipo_promo_3,...,avg_l3m_total_categoria_2,avg_l3m_total_categoria_3,avg_l3m_total_categoria_4,avg_l3m_total_categoria_5,avg_l3m_total_categoria_6,avg_l3m_total_categoria_7,avg_l3m_total_departamento_0,avg_l3m_total_compras_mes,target,date
0,18,597.210022,30.0,0.0,0.00,0.0,0.0,0.0,0.0,31.0,...,0.0,0.0,1.0,9.0,4.0,2.0,27.0,31,0,2021-12-01
1,65,189.869995,24.0,3.0,1.72,0.0,1.0,0.0,2.0,20.0,...,10.0,15.0,20.0,9.0,4.0,1.0,29.0,76,0,2021-12-01
2,515,158.580002,21.0,0.0,0.00,0.0,0.0,0.0,0.0,15.0,...,0.0,3.0,4.0,8.0,0.0,0.0,8.0,15,0,2021-12-01
3,1092,1395.300049,103.0,2.0,10.48,0.0,2.0,0.0,0.0,54.0,...,22.0,27.0,19.0,35.0,41.0,3.0,90.0,162,0,2021-12-01
4,1476,1397.280029,85.0,0.0,0.00,0.0,0.0,0.0,0.0,57.0,...,8.0,9.0,14.0,20.0,28.0,0.0,98.0,130,0,2021-12-01


In [30]:
len(samples_df)

171937

In [31]:
pos_neg_ratio = len(samples_df[samples_df['target']==1])/len(samples_df)
pos_neg_ratio

0.026236354013388623

In [32]:
# samples_df.to_parquet('gs://gpa-churn/data/processed/sales/sales_100k.parquet')

---

### Merging with user data (register)

In [33]:
register = get_register_data(tables_dict)
register = register[register['data_nascimento']>='1900']

added file:  STAGING/tck_origemcadastro_20220425.zip
added file:  STAGING/tck_origemcadastro_20220427.zip
added file:  STAGING/tck_origemcadastro_20220428.zip
added file:  STAGING/tck_origemcadastro_20220429.zip
added file:  STAGING/tck_origemcadastro_20220430.zip
added file:  STAGING/tck_origemcadastro_20220501.zip
added file:  STAGING/tck_origemcadastro_20220502.zip
added file:  STAGING/tck_origemcadastro_20220503.csv.zip
added file:  STAGING/tck_origemcadastro_20220504.csv.zip
added file:  STAGING/tck_origemcadastro_20220505.csv.zip
added file:  STAGING/tck_origemcadastro_20220506.csv.zip
added file:  STAGING/tck_origemcadastro_20220509.csv.zip
added file:  STAGING/tck_origemcadastro_20220510.csv.zip
----------


In [34]:
def get_region(x):    
    region_dict = {
        'se':['sp','mg','rj','es'],  #Sudeste
        's':['pr','sc','rs'],        #Sul
        'ne':['ma','ce','rn','pb','pe','al','se','ba','pi'], #Nordeste
        'n':['rr','ap','am','pa','ac','ro','to'],   #Norte
        'co':['mt','df','go','ms']    #Centro Oeste
        }
    
    for key in region_dict.keys():
        try:
            if x in region_dict[key]:
                return key
        except:
            pass
              
    return 'null'


def get_pib_from_uf(x):
    
    pib_dict = {
        'df':90742,
        'sp':51140,
        'rj':45174,
        'sc':45118,
        'rs':42406,
        'pr':40788,
        'mt':40787,
        'ms':38482,
        'es':34177,
        'mg':30794,
        'go':29732,
        'ro':26497,
        'am':26101,
        'to':25022,
        'rr':23594,
        'pa':20734,
        'pe':20702,
        'ap':20688,
        'rn':20342,
        'ba':19716,
        'se':19441,
        'ce':17912,
        'ac':17722,
        'al':17667,
        'pb':16919,
        'pi':16125,
        'ma':13757
    }
    
    try:
        return pib_dict[x]
    except:
        return 'null'


geolocator = Nominatim(user_agent="get_lat_long", timeout=1)
def get_lat_long(municipio):

    location = geolocator.geocode(municipio)
    return location.latitude, location.longitude


register['region'] = register['uf'].apply(lambda x:get_region(x))
register['pib_percapita'] = register['uf'].apply(lambda x:get_pib_from_uf(x))
#register['lat'], register['long'] = zip(*register['uf'].map(get_lat_long))
register.head()

,cod_cliente,data_cadastro,data_registro,sexo,cidade,uf,data_nascimento,region,pib_percapita
0,1,2000-08-10,2014-12-30,F,fortaleza,ce,1970-12-29,ne,17912
1,14,2020-05-27,2020-05-27,F,brasilia,df,1953-06-26,co,90742
2,15,2000-08-10,2008-05-24,F,fortaleza,ce,1920-11-11,ne,17912
3,16,2000-08-10,2008-05-24,M,fortaleza,ce,1942-02-25,ne,17912
4,17,2000-08-10,2008-05-23,M,fortaleza,ce,1920-11-11,ne,17912


In [35]:
samples_df = pd.merge(samples_df, register, on=['cod_cliente'], how='left')
del register
gc.collect()
samples_df

,cod_cliente,total_val_venda_bruta_cupom,total_qtd_item_venda,total_flg_vend_meu_desct,total_valor_desconto,total_flag_dev,total_tipo_promo_0,total_tipo_promo_1,total_tipo_promo_2,total_tipo_promo_3,...,target,date,data_cadastro,data_registro,sexo,cidade,uf,data_nascimento,region,pib_percapita
0,18,597.210022,30.0,0.0,0.000000,0.0,0.0,0.0,0.0,31.0,...,0,2021-12-01,2000-08-10,2009-08-07,M,fortaleza,ce,1920-11-11,ne,17912.0
1,65,189.869995,24.0,3.0,1.720000,0.0,1.0,0.0,2.0,20.0,...,0,2021-12-01,2000-08-10,2022-01-11,F,fortaleza,ce,1968-01-08,ne,17912.0
2,515,158.580002,21.0,0.0,0.000000,0.0,0.0,0.0,0.0,15.0,...,0,2021-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1092,1395.300049,103.0,2.0,10.480000,0.0,2.0,0.0,0.0,54.0,...,0,2021-12-01,2007-07-13,2022-02-28,F,santana de parnaiba,sp,1963-05-16,se,51140.0
4,1476,1397.280029,85.0,0.0,0.000000,0.0,0.0,0.0,0.0,57.0,...,0,2021-12-01,2000-12-22,2021-12-21,F,itu,sp,1956-09-02,se,51140.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171938,54191460,415.799988,75.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0,2022-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171939,54191464,319.860016,37.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0,2022-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171940,54191472,1148.650024,84.0,19.0,28.800001,0.0,0.0,2.0,17.0,36.0,...,0,2022-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171941,54191496,395.059998,44.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0,2022-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# samples_df = samples_df[samples_df['date']>=samples_df['data_registro']]
samples_df = samples_df.sort_values(by=['cod_cliente','data_registro'], ascending=True)
samples_df = samples_df.drop_duplicates(subset=['cod_cliente','date'],keep='last')
samples_df

,cod_cliente,total_val_venda_bruta_cupom,total_qtd_item_venda,total_flg_vend_meu_desct,total_valor_desconto,total_flag_dev,total_tipo_promo_0,total_tipo_promo_1,total_tipo_promo_2,total_tipo_promo_3,...,target,date,data_cadastro,data_registro,sexo,cidade,uf,data_nascimento,region,pib_percapita
0,18,597.210022,30.0,0.0,0.000000,0.0,0.0,0.0,0.0,31.0,...,0,2021-12-01,2000-08-10,2009-08-07,M,fortaleza,ce,1920-11-11,ne,17912.0
86228,18,11.590000,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,2.0,...,0,2022-01-01,2000-08-10,2009-08-07,M,fortaleza,ce,1920-11-11,ne,17912.0
1,65,189.869995,24.0,3.0,1.720000,0.0,1.0,0.0,2.0,20.0,...,0,2021-12-01,2000-08-10,2022-01-11,F,fortaleza,ce,1968-01-08,ne,17912.0
86229,65,395.359985,32.0,6.0,6.190000,0.0,3.0,0.0,2.0,31.0,...,0,2022-01-01,2000-08-10,2022-01-11,F,fortaleza,ce,1968-01-08,ne,17912.0
2,515,158.580002,21.0,0.0,0.000000,0.0,0.0,0.0,0.0,15.0,...,0,2021-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171938,54191460,415.799988,75.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0,2022-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171939,54191464,319.860016,37.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0,2022-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171940,54191472,1148.650024,84.0,19.0,28.800001,0.0,0.0,2.0,17.0,36.0,...,0,2022-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171941,54191496,395.059998,44.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0,2022-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
samples_df['data_nascimento'] = pd.to_datetime(samples_df['data_nascimento'])
samples_df['idade'] = samples_df['date']-samples_df['data_nascimento']
samples_df['idade'] = samples_df['idade'].apply(lambda x:x.days)
samples_df

,cod_cliente,total_val_venda_bruta_cupom,total_qtd_item_venda,total_flg_vend_meu_desct,total_valor_desconto,total_flag_dev,total_tipo_promo_0,total_tipo_promo_1,total_tipo_promo_2,total_tipo_promo_3,...,date,data_cadastro,data_registro,sexo,cidade,uf,data_nascimento,region,pib_percapita,idade
0,18,597.210022,30.0,0.0,0.000000,0.0,0.0,0.0,0.0,31.0,...,2021-12-01,2000-08-10,2009-08-07,M,fortaleza,ce,1920-11-11,ne,17912.0,36910.0
86228,18,11.590000,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,2.0,...,2022-01-01,2000-08-10,2009-08-07,M,fortaleza,ce,1920-11-11,ne,17912.0,36941.0
1,65,189.869995,24.0,3.0,1.720000,0.0,1.0,0.0,2.0,20.0,...,2021-12-01,2000-08-10,2022-01-11,F,fortaleza,ce,1968-01-08,ne,17912.0,19686.0
86229,65,395.359985,32.0,6.0,6.190000,0.0,3.0,0.0,2.0,31.0,...,2022-01-01,2000-08-10,2022-01-11,F,fortaleza,ce,1968-01-08,ne,17912.0,19717.0
2,515,158.580002,21.0,0.0,0.000000,0.0,0.0,0.0,0.0,15.0,...,2021-12-01,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171938,54191460,415.799988,75.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,2022-01-01,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
171939,54191464,319.860016,37.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,2022-01-01,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
171940,54191472,1148.650024,84.0,19.0,28.800001,0.0,0.0,2.0,17.0,36.0,...,2022-01-01,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
171941,54191496,395.059998,44.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,2022-01-01,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN


In [38]:
samples_df['data_cadastro'] = pd.to_datetime(samples_df['data_cadastro'])
samples_df['delta_de_cadastro'] = samples_df['date']-samples_df['data_cadastro']
samples_df['delta_de_cadastro'] = samples_df['delta_de_cadastro'].apply(lambda x:x.days)
samples_df

,cod_cliente,total_val_venda_bruta_cupom,total_qtd_item_venda,total_flg_vend_meu_desct,total_valor_desconto,total_flag_dev,total_tipo_promo_0,total_tipo_promo_1,total_tipo_promo_2,total_tipo_promo_3,...,data_cadastro,data_registro,sexo,cidade,uf,data_nascimento,region,pib_percapita,idade,delta_de_cadastro
0,18,597.210022,30.0,0.0,0.000000,0.0,0.0,0.0,0.0,31.0,...,2000-08-10,2009-08-07,M,fortaleza,ce,1920-11-11,ne,17912.0,36910.0,7783.0
86228,18,11.590000,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,2.0,...,2000-08-10,2009-08-07,M,fortaleza,ce,1920-11-11,ne,17912.0,36941.0,7814.0
1,65,189.869995,24.0,3.0,1.720000,0.0,1.0,0.0,2.0,20.0,...,2000-08-10,2022-01-11,F,fortaleza,ce,1968-01-08,ne,17912.0,19686.0,7783.0
86229,65,395.359985,32.0,6.0,6.190000,0.0,3.0,0.0,2.0,31.0,...,2000-08-10,2022-01-11,F,fortaleza,ce,1968-01-08,ne,17912.0,19717.0,7814.0
2,515,158.580002,21.0,0.0,0.000000,0.0,0.0,0.0,0.0,15.0,...,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171938,54191460,415.799988,75.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
171939,54191464,319.860016,37.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
171940,54191472,1148.650024,84.0,19.0,28.800001,0.0,0.0,2.0,17.0,36.0,...,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
171941,54191496,395.059998,44.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


### Merging with data of discount programs (stix)

In [39]:
stix = get_stix_data(tables_dict)
stix

added file:  STAGING/v_stix_optin_20220427.zip
added file:  STAGING/v_stix_optin_20220428.zip
added file:  STAGING/v_stix_optin_20220429.zip
added file:  STAGING/v_stix_optin_20220430.zip
added file:  STAGING/v_stix_optin_20220501.zip
added file:  STAGING/v_stix_optin_20220502.zip
added file:  STAGING/v_stix_optin_20220503.csv.zip
added file:  STAGING/v_stix_optin_20220504.csv.zip
added file:  STAGING/v_stix_optin_20220505.csv.zip
added file:  STAGING/v_stix_optin_20220506.csv.zip
added file:  STAGING/v_stix_optin_20220509.csv.zip
added file:  STAGING/v_stix_optin_20220510.csv.zip
----------


,cod_cliente,data_registro_stix,ind_email
0,22,2020-07-03,1
1,40,2021-03-10,1
2,64,2020-07-29,1
3,65,2020-10-14,1
4,78,2020-12-20,1
...,...,...,...
2860846,54595171,2022-05-09,0
2860847,54595179,2022-02-10,0
2860848,54595183,2022-05-09,1
2860849,54595192,2022-05-09,1


In [40]:
samples_df = pd.merge(samples_df, stix, on=['cod_cliente'], how='left', indicator=True)
del stix
gc.collect()
samples_df

,cod_cliente,total_val_venda_bruta_cupom,total_qtd_item_venda,total_flg_vend_meu_desct,total_valor_desconto,total_flag_dev,total_tipo_promo_0,total_tipo_promo_1,total_tipo_promo_2,total_tipo_promo_3,...,cidade,uf,data_nascimento,region,pib_percapita,idade,delta_de_cadastro,data_registro_stix,ind_email,_merge
0,18,597.210022,30.0,0.0,0.000000,0.0,0.0,0.0,0.0,31.0,...,fortaleza,ce,1920-11-11,ne,17912.0,36910.0,7783.0,NaN,NaN,left_only
1,18,11.590000,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,2.0,...,fortaleza,ce,1920-11-11,ne,17912.0,36941.0,7814.0,NaN,NaN,left_only
2,65,189.869995,24.0,3.0,1.720000,0.0,1.0,0.0,2.0,20.0,...,fortaleza,ce,1968-01-08,ne,17912.0,19686.0,7783.0,2020-10-14,1.0,both
3,65,395.359985,32.0,6.0,6.190000,0.0,3.0,0.0,2.0,31.0,...,fortaleza,ce,1968-01-08,ne,17912.0,19717.0,7814.0,2020-10-14,1.0,both
4,515,158.580002,21.0,0.0,0.000000,0.0,0.0,0.0,0.0,15.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171932,54191460,415.799988,75.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
171933,54191464,319.860016,37.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
171934,54191472,1148.650024,84.0,19.0,28.800001,0.0,0.0,2.0,17.0,36.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,2022-01-02,0.0,both
171935,54191496,395.059998,44.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [41]:
samples_df.rename(columns={'_merge':'cadastro_stix'}, inplace=True)
samples_df['cadastro_stix'] = [0 if i=='left_only' else 1 for i in list(samples_df['cadastro_stix'])]
samples_df

,cod_cliente,total_val_venda_bruta_cupom,total_qtd_item_venda,total_flg_vend_meu_desct,total_valor_desconto,total_flag_dev,total_tipo_promo_0,total_tipo_promo_1,total_tipo_promo_2,total_tipo_promo_3,...,cidade,uf,data_nascimento,region,pib_percapita,idade,delta_de_cadastro,data_registro_stix,ind_email,cadastro_stix
0,18,597.210022,30.0,0.0,0.000000,0.0,0.0,0.0,0.0,31.0,...,fortaleza,ce,1920-11-11,ne,17912.0,36910.0,7783.0,NaN,NaN,0
1,18,11.590000,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,2.0,...,fortaleza,ce,1920-11-11,ne,17912.0,36941.0,7814.0,NaN,NaN,0
2,65,189.869995,24.0,3.0,1.720000,0.0,1.0,0.0,2.0,20.0,...,fortaleza,ce,1968-01-08,ne,17912.0,19686.0,7783.0,2020-10-14,1.0,1
3,65,395.359985,32.0,6.0,6.190000,0.0,3.0,0.0,2.0,31.0,...,fortaleza,ce,1968-01-08,ne,17912.0,19717.0,7814.0,2020-10-14,1.0,1
4,515,158.580002,21.0,0.0,0.000000,0.0,0.0,0.0,0.0,15.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171932,54191460,415.799988,75.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0
171933,54191464,319.860016,37.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0
171934,54191472,1148.650024,84.0,19.0,28.800001,0.0,0.0,2.0,17.0,36.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,2022-01-02,0.0,1
171935,54191496,395.059998,44.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0


In [43]:
samples_df = samples_df.sort_values(by=['cod_cliente','data_registro_stix'], ascending=True)
samples_df = samples_df.drop_duplicates(subset=['cod_cliente','date'],keep='last')
samples_df

,cod_cliente,total_val_venda_bruta_cupom,total_qtd_item_venda,total_flg_vend_meu_desct,total_valor_desconto,total_flag_dev,total_tipo_promo_0,total_tipo_promo_1,total_tipo_promo_2,total_tipo_promo_3,...,cidade,uf,data_nascimento,region,pib_percapita,idade,delta_de_cadastro,data_registro_stix,ind_email,cadastro_stix
0,18,597.210022,30.0,0.0,0.000000,0.0,0.0,0.0,0.0,31.0,...,fortaleza,ce,1920-11-11,ne,17912.0,36910.0,7783.0,NaN,NaN,0
1,18,11.590000,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,2.0,...,fortaleza,ce,1920-11-11,ne,17912.0,36941.0,7814.0,NaN,NaN,0
2,65,189.869995,24.0,3.0,1.720000,0.0,1.0,0.0,2.0,20.0,...,fortaleza,ce,1968-01-08,ne,17912.0,19686.0,7783.0,2020-10-14,1.0,1
3,65,395.359985,32.0,6.0,6.190000,0.0,3.0,0.0,2.0,31.0,...,fortaleza,ce,1968-01-08,ne,17912.0,19717.0,7814.0,2020-10-14,1.0,1
4,515,158.580002,21.0,0.0,0.000000,0.0,0.0,0.0,0.0,15.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171932,54191460,415.799988,75.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0
171933,54191464,319.860016,37.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0
171934,54191472,1148.650024,84.0,19.0,28.800001,0.0,0.0,2.0,17.0,36.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,2022-01-02,0.0,1
171935,54191496,395.059998,44.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0


In [44]:
samples_df['data_registro_stix'] = pd.to_datetime(samples_df['data_registro_stix'])
samples_df['delta_de_stix'] = samples_df['date']-samples_df['data_registro_stix']
samples_df['delta_de_stix'] = samples_df['delta_de_stix'].apply(lambda x:x.days)
samples_df

,cod_cliente,total_val_venda_bruta_cupom,total_qtd_item_venda,total_flg_vend_meu_desct,total_valor_desconto,total_flag_dev,total_tipo_promo_0,total_tipo_promo_1,total_tipo_promo_2,total_tipo_promo_3,...,uf,data_nascimento,region,pib_percapita,idade,delta_de_cadastro,data_registro_stix,ind_email,cadastro_stix,delta_de_stix
0,18,597.210022,30.0,0.0,0.000000,0.0,0.0,0.0,0.0,31.0,...,ce,1920-11-11,ne,17912.0,36910.0,7783.0,NaT,NaN,0,NaN
1,18,11.590000,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,2.0,...,ce,1920-11-11,ne,17912.0,36941.0,7814.0,NaT,NaN,0,NaN
2,65,189.869995,24.0,3.0,1.720000,0.0,1.0,0.0,2.0,20.0,...,ce,1968-01-08,ne,17912.0,19686.0,7783.0,2020-10-14,1.0,1,413.0
3,65,395.359985,32.0,6.0,6.190000,0.0,3.0,0.0,2.0,31.0,...,ce,1968-01-08,ne,17912.0,19717.0,7814.0,2020-10-14,1.0,1,444.0
4,515,158.580002,21.0,0.0,0.000000,0.0,0.0,0.0,0.0,15.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171932,54191460,415.799988,75.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,0,NaN
171933,54191464,319.860016,37.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,0,NaN
171934,54191472,1148.650024,84.0,19.0,28.800001,0.0,0.0,2.0,17.0,36.0,...,NaN,NaT,NaN,NaN,NaN,NaN,2022-01-02,0.0,1,-1.0
171935,54191496,395.059998,44.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,0,NaN


In [46]:
samples_df.to_parquet('gs://gpa-churn/data/processed/steps/after_stix_100k.parquet', compression='gzip')